In [ ]:
options("scipen"=100, "digits"=4)
if(!require("readr")) install.packages("readr")
if(!require("rpart")) install.packages("rpart")
if(!require("rpart.plot")) install.packages("rpart.plot")
if(!require("Metrics")) install.packages("Metrics")
library("readr")
library("rpart")
library("rpart.plot")
library("Metrics")

Building the Regression Tree Model
----------------------------------

First lets take a look at the data we will work with. Suppose we have
the following data from 15 months of demand for some product and want to
predict the demand based on whether there was a markdown or not and also
whether or not there was a promotion.

So here is the data we have, this will be our training data:

| markdown | promotion | demand |
|----------|-----------|--------|
| yes      | yes       | 74     |
| no       | no        | 23     |
| yes      | yes       | 61     |
| yes      | yes       | 59     |
| no       | no        | 25     |
| yes      | yes       | 63     |
| no       | yes       | 54     |
| yes      | no        | 42     |
| no       | yes       | 55     |
| yes      | yes       | 75     |
| no       | no        | 13     |
| yes      | yes       | 73     |
| yes      | no        | 31     |
| no       | no        | 12     |
| no       | no        | 11     |

-   `demand` is our result or outcome
-   `markdown` (true or false), `promotion` (true or false) are the
    predictors

We hope that these predictors will help up predict the `demand`.

To start we read the training data above from a csv file and print out
its structore:

In [ ]:
#trainurl<-"promotions.csv"
trainurl<-"https://docs.google.com/spreadsheets/d/e/2PACX-1vTT6LcnhxE-mij5m5HHdLZ77YB1_teFiPUrM2uyafkGMUHs-zSKgylLnvgEOleRCB6jatqV1JjAczkp/pub?gid=0&single=true&output=csv"
traindf<-read.csv(trainurl, stringsAsFactors=TRUE)
str(traindf)

Let’s print out the training set to make sure it looks like what we
expect:

In [ ]:
print(traindf)

We will try to understand how to build up a decision tree for this
example by just using one variable at a time to model the decision tree.

### Splitting on Promotion

First lets take a look at a simple model that uses just `promotion` to
try to predict `demand`.

Suppose we decide to split on `promotion`. What will the leaf nodes look
like? Since there are just 2 values for promotion, we can group all the
rows where `promotion` is “yes” together and all the rows where
`promotion` is “no”.

Here are the rows for leaf1, where `promotion` is “yes”

In [ ]:
leaf1<-traindf[traindf$promotion == "yes",]
print(leaf1)

And here are the rows for leaf2, where `promotion` is “no”

In [ ]:
leaf2<-traindf[traindf$promotion == "no",]
print(leaf2)

Lets look at the mean demand and standard deviation from the entire
training set, then compare that to the mean and the standard deviation
in each leaf node. The question here is if each leaf node is more pure
given that we perform this split on `promotion`.

In [ ]:
compare<-data.frame(train= c(mean(traindf$demand), sd(traindf$demand), nrow(traindf)), 
            leaf1= c(mean(leaf1$demand), sd(leaf1$demand), nrow(leaf1)), 
            leaf2= c(mean(leaf2$demand), sd(leaf2$demand), nrow(leaf2)))
row.names(compare) <- c("mean", "std", "size")
print(compare)

Next lets take the weighted average of the std deviation in leaf1 and
the std deviation in leaf2.

In [ ]:
n<- compare[3,1]
n1<-compare[3,2]
n2<-compare[3,3]
sd1<-compare[2,2]
sd2<-compare[2,3]
# Take a weighted average of the std deviations of leaf1 and leaf2
newsd <- (n1/n)*sd1 + (n2/n)*sd2 
newsd 

So we are ready to see the predictions in each leaf node:

In [ ]:
control <- rpart.control(minbucket=1, maxdepth=1)
model1 <- rpart(demand~promotion, data=traindf, method="anova", control=control)
rpart.plot(model1, type=4, extra=101)

Notice that the prediction in each leaf node is just the mean over the
rows that wind up in that leaf node.

### Splitting on Markdown

Next lets look at a model that uses just `markdown` to try to predict
`demand`:

Here are the rows for leaf1, where `markdown` is “yes”

In [ ]:
leaf1<-traindf[traindf$markdown == "yes",]
print(leaf1)

And here are the rows for leaf2, where `markdown` is “no”

In [ ]:
leaf2<-traindf[traindf$markdown == "no",]
print(leaf2)

Here is the comparison of the entire training data and the std
deviations in each node if we split on `markdown`:

In [ ]:
compare<-data.frame(train= c(mean(traindf$demand), sd(traindf$demand), nrow(traindf)), 
            leaf1= c(mean(leaf1$demand), sd(leaf1$demand), nrow(leaf1)), 
            leaf2= c(mean(leaf2$demand), sd(leaf2$demand), nrow(leaf2)))
row.names(compare) <- c("mean", "std", "size")
print(compare)

Once again we take the weighted average of the std deviation in leaf1
and the std deviation in leaf2.

In [ ]:
n<- compare[3,1]
n1<-compare[3,2]
n2<-compare[3,3]
sd1<-compare[2,2]
sd2<-compare[2,3]
# Take a weighted average of the std deviations of leaf1 and leaf2
newsd <- (n1/n)*sd1 + (n2/n)*sd2 
newsd 

Now lets do the model using just `markdown` and one node to predict
`demand`:

In [ ]:
control <- rpart.control(minbucket=1, maxdepth=1)
model2 <- rpart(demand~markdown, data=traindf, method="anova", control=control)
rpart.plot(model2, type=4, extra=101)

### Regression Tree built using both `markdown` and `promotion`

What about if we use both `promotion` and `markdown` together to predict
`demand`:

In [ ]:
control <- rpart.control(minbucket=1)
model3 <- rpart(demand~markdown+promotion, data=traindf, method="anova", control=control)
rpart.plot(model3, type=4, extra=101)

Testing the Regression Tree
---------------------------

Next let’s look at some data that we can use as a testing set. This data
we will use to evaluate how well the models we looked at above will do
when they see “new” data. This gives a better idea of how accurate our
model is. So here is the data we will use as our testing data:

First we read the test set and look at its structure:

In [ ]:
# testurl<-"promotions-test.csv"
testurl<-"https://docs.google.com/spreadsheets/d/e/2PACX-1vTT6LcnhxE-mij5m5HHdLZ77YB1_teFiPUrM2uyafkGMUHs-zSKgylLnvgEOleRCB6jatqV1JjAczkp/pub?gid=1414619388&single=true&output=csv"
testdf<-read.csv(testurl, stringsAsFactors=TRUE)
str(testdf)

And now print out the test set:

In [ ]:
print(testdf)

### Testing `demand~markdown+promotion`

Now we are ready to do some predictions using the test dataframe:

In [ ]:
pred <- predict(model3, newdata = testdf)
str(pred)

So `pred` now holds our predictions.

We can now compare those predictions (from our model) with the actual
known results from the testing data. Here we are comparing how our model
“predicted” on the test data with the “actual” outcomes that were
included in the testing data. Here it is important that we know the
actual outcomes on the testing data so we can see how we do.

Lets make a data frame that shows the prediction and the actuals:

In [ ]:
comparedf <- data.frame(actual=testdf$demand, predictions=pred)
print(comparedf)

Here is the MAPE, MAE and RMSE for our best model (model3):

In [ ]:
mape(testdf$demand, pred)
mae(testdf$demand, pred)
rmse(testdf$demand, pred)

### Testing `demand~promotion`

Here is the actual and the predicted for model1:

In [ ]:
pred <- predict(model1, newdata = testdf)
comparedf <- data.frame(actual=testdf$demand, predictions=pred) 
print(comparedf)

Here is the MAPE, MAE and RMSE for model1:

In [ ]:
mape(testdf$demand, pred)
mae(testdf$demand, pred)
rmse(testdf$demand, pred)

### Testing `demand~markdown`

Here is the actual and the predicted for model2:

In [ ]:
pred <- predict(model2, newdata = testdf)
comparedf <- data.frame(actual=testdf$demand, predictions=pred)
print(comparedf)

Here is the MAPE, MAE and RMSE for model2:

In [ ]:
mape(testdf$demand, pred)
mae(testdf$demand, pred)
rmse(testdf$demand, pred)